In [1]:
import pandas as pd
import mysql.connector
from faker import Faker
import random

In [2]:
# Initialize Faker
fake = Faker()

In [3]:
# Expense categories and payment modes
categories = ["Groceries", "Travel", "Transportation", "Entertainment", "Gifts", "Income Tax", "Gas Bill", "Subscriptions"]
payment_modes = ["Cash", "Credit Card", "Debit Card", "UPI"]

In [4]:
# Category-specific descriptions
description_map = {
    "Groceries": "Purchased groceries and household items.",
    "Travel": "Expense for travel tickets and accommodation.",
    "Transportation": "Daily commute and fuel expenses.",
    "Entertainment": "Movies, concerts, and recreational activities.",
    "Gifts": "Purchased gifts for friends and family.",
    "Income Tax": "Annual income tax payment.",
    "Gas Bill": "Monthly gas bill payment.",
    "Subscriptions": "Online streaming and magazine subscriptions."
}

In [5]:
# Function to generate expenses
def generate_expenses():
    data = []
    for month in range(1, 13):  # January to December
        count = 0
        gas_bill_paid = False  # Ensure Gas Bill is paid only once per month
        while count < 150:  # 150 records per month
            date = fake.date_between_dates(date_start=pd.Timestamp(f"2024-{month:02d}-01"), 
                                           date_end=pd.Timestamp(f"2024-{month:02d}-28")).strftime("%Y-%m-%d")
            category = random.choice(categories)
            
            # Ensure Income Tax is only in March
            if category == "Income Tax" and month != 3:
                continue
            
            # Ensure Gas Bill is only paid once per month
            if category == "Gas Bill":
                if gas_bill_paid:
                    continue
                gas_bill_paid = True
            
            payment_mode = random.choice(payment_modes)
            amount_paid = round(random.uniform(100, 5000), 2)
            cashback = round(random.uniform(1, 20), 2) if payment_mode == "UPI" else 0
            description = description_map[category]
            
            data.append([date, category, payment_mode, description, amount_paid, cashback])
            count += 1
    
    return pd.DataFrame(data, columns=["Date", "Category", "Payment Mode", "Description", "Amount Paid", "Cashback"])


In [6]:
# Generate DataFrame
df = generate_expenses()

In [7]:
# Save to CSV
df.to_csv("monthly_expenses_2024.csv", index=False)
print("CSV file created successfully.")

CSV file created successfully.


In [8]:
# MySQL connection details
mysql_config = {
    "host": "localhost",
    "user": "root",
    "password": "Ashok@mysql123",
    "database": "expense_db"
}

In [9]:
# Connect to MySQL and create database
conn = mysql.connector.connect(host=mysql_config["host"], user=mysql_config["user"], password=mysql_config["password"])
cursor = conn.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS expense_db")
cursor.close()
conn.close()

In [10]:
# Connect to expense_db
conn = mysql.connector.connect(**mysql_config)
cursor = conn.cursor()


In [11]:
# Create table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS expenses (
        id INT AUTO_INCREMENT PRIMARY KEY,
        date DATE,
        category VARCHAR(50),
        payment_mode VARCHAR(20),
        description TEXT,
        amount_paid DECIMAL(10,2),
        cashback DECIMAL(5,2)
    )
""")

In [12]:
# Insert data into MySQL
def insert_into_mysql(df):
    for _, row in df.iterrows():
        cursor.execute("""
            INSERT INTO expenses (date, category, payment_mode, description, amount_paid, cashback)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, tuple(row))
    conn.commit()
    print("Data inserted into MySQL successfully.")

In [13]:
# Insert DataFrame into MySQL
insert_into_mysql(df)

Data inserted into MySQL successfully.


In [14]:
# Close connection
cursor.close()
conn.close()
print("Database connection closed.")


Database connection closed.
